In [1]:
from sklearn import datasets
from sklearn.cluster import AgglomerativeClustering
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np

dic = {}
df = pd.read_csv('./chicago-west-nile-virus-mosquito-test-results/west-nile-virus-wnv-mosquito-test-results.csv', encoding="utf-8")
df = df.drop(columns = 'LOCATION')
df['TEST DATE'] = df['TEST DATE'].str.split('T').str[0].str.replace('-', '')
nonNumCol = ['TRAP', 'BLOCK', 'TRAP_TYPE', 'RESULT', 'SPECIES']
for col in nonNumCol:
    df[col] = df[col].astype('category')
    dic[col] = dict(enumerate(df[col].cat.categories))
    df[col] = df[col].cat.codes
df = df.dropna()
data = df.to_numpy()

# clustering3 = AgglomerativeClustering(linkage = 'single',  n_clusters = 10)
# result = clustering3.fit(data)

In [ ]:
iris = datasets.load_iris()
irisdata = iris.data

clustering = AgglomerativeClustering(linkage='ward', n_clusters=3)

res = clustering.fit(irisdata)

print ("各個簇的樣本數目：")
print (pd.Series(clustering.labels_).value_counts())
print ("聚類結果：")
print (confusion_matrix(iris.target, clustering.labels_))

plt.figure()
d0 = irisdata[clustering.labels_ == 0]
plt.plot(d0[:, 0], d0[:, 1], 'r.')
d1 = irisdata[clustering.labels_ == 1]
plt.plot(d1[:, 0], d1[:, 1], 'go')
d2 = irisdata[clustering.labels_ == 2]
plt.plot(d2[:, 0], d2[:, 1], 'b*')
plt.xlabel("Sepal.Length")
plt.ylabel("Sepal.Width")
plt.title("AGNES Clustering")
plt.show()